In [ ]:
import pandas as pd
from datasets import Dataset

desc_template = """Frame Name: {frame_name}
Frame Definition: {frame_definition}

Frame Elements:
{frame_elements}
"""

system_prompt_template = """### Task:
You are given a sentence in Chinese and a frame with its associated frame elements. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word (surrounded by **). 

### Notes:
- Return the tagged sentence in a ```json ``` code block.
- Frame elements must never overlap.
- Return your answer in Chinese.
"""

user_prompt_template = """### Frame Information:
{frame_info}

### Input:
{input_sentence}
"""

assistant_prompt_template = """### Output: 
```json
{output_sentence}
```
"""

def highlight_target_words(row):
    targets = sorted([(x['start'], x['end']+1) for x in row['target']])
    merged_target = (targets[0][0], targets[-1][-1])
    return f"{row['text'][:merged_target[0]]}**{row['text'][merged_target[0]:merged_target[1]]}**{row['text'][merged_target[1]:]}" 

def get_expected_output(row):
    fe_spans = sorted([(x['fe_name'], x['start'], x['end']+1) for x in row['cfn_spans']], key=lambda x: x[1])
    fe_spans = {x[0]: row['sentence'][x[1]:x[2]] for x in fe_spans}
    return str(fe_spans)



def create_llm_messages(row, frame_info):
    input_sentence = row['input_sentence']
    frame_description = frame_info.query(f"frame_name == '{row['frame_name']}'")['description'].values[0]

    user_prompt = user_prompt_template.format(
        frame_info=frame_description,
        input_sentence=input_sentence
    )

    assistant_prompt = assistant_prompt_template.format(
        output_sentence=row['output_sentence']
    )



def create_description(row):
    return desc_template.format(
        frame_name=row['frame_name'],
        frame_definition=row['frame_def'],
        frame_elements='\n'.join([f"{x['fe_name']}: {x['fe_def']}" for x in row['fes']])
    )

# train_prompts_df = pd.read_json('../fn1.7-train-prompts.jsonl', lines=True)
# train_prompts = Dataset.from_pandas(train_prompts_df)

cfn_frame_info = pd.read_json('/mnt/d/Projects/llm-fsp/data/raw/cfn-dataset/frame_info.json')
cfn_frame_info['description'] = cfn_frame_info.apply(create_description, axis=1)

cfn_train = pd.read_json('/mnt/d/Projects/llm-fsp/data/raw/cfn-dataset/cfn-train.json')
cfn_train['input_sentence'] = cfn_train.apply(highlight_target_words, axis=1)


In [48]:
cfn_train.apply(get_expected_output, axis=1)

TypeError: tuple indices must be integers or slices, not str

In [23]:
cfn_frame_info.fes.apply(len).describe()

count    713.000000
mean      42.737728
std        2.470601
min       38.000000
25%       41.000000
50%       42.000000
75%       44.000000
max       54.000000
Name: fes, dtype: float64

In [20]:
print(cfn_frame_info.description.values[5])

Frame Name: 发声
Frame Definition: 一个物理实体作为声源，发出一个声响；声源可以是无生体，也可以是动物或人，后者用声道发出声音；有时用一个名词性成分表示声音本身，成为声响；同时，该框架也可以出现表示修饰的词语，这些词语描述声响的特征。

Frame Elements:
有声事件: 一个或多个声源参与的伴随着声响的事件。
声响: 声响元素有时隐含在该框架所包含动词词元的语义中，有时则由单独的名词性成分充当，后者标注为声响。
声源: 表示发出声音的物理实体，这个实体必须有比较复杂的内部结构，以使它能够发出声音，例如，一个带有可移动的部件的机器或有声带的动物、人，能靠自身发出声音。
信息接收者: 信息接收者表示听者，他听到所产生的声音。
方向: 表示声源朝哪里或朝谁发出声音，注意该元素有时和信息接收者容易产生混淆，但是信息接收者元素不能做介词短语的宾语，而方向需要用介词短语表示。
程度: 事件发生的程度。
描述: 在行为过程中声源所处的状态。
声源位置: 声源所处的位置，声响从其中发出。
方式: 描述声音的特征，如：吵闹地，刺耳地，等等。
路径: 声响沿着路径从声源处发出。
处所: 声源所在的处所。该框架元素和声源位置只是叙述视角不同，实际上指的是同一种情况，二者有明显的语序差别：表示空间的词语在声源之前，表示声源位置的词语在声源之后，"观众席中发出惊叹声"这样的没有"声源"出现的句子，表示处所的状语仍为声源位置。
亚区: 亚区是声源的局部区域，声音从中发出。
时间: 有声事件发生的时间。
角色事件时间: 表示目标词所涉及的事件发生的时间，而不是目标词本身所表示的动作行为发生的时间。
动作时量: 动作行为本身持续的时间数量或者动作重复进行所用的时间数量。
结果时量: 动作行为造成的结果状态所持续的时间长度
频率: 表示事件在单位时间内所发生的次数，一般可以用（“每”＋时间单位＋“多少次”）提问。
次数: 表示事件或者状态重复发生的次数。
特定次数: 该框架元素表示，单独看目标词所在句子，并没有动作的重复，但把目标词置于上下文中，则认为目标词所表示的动作是重复前面的动作；还有可能仅仅是假想目标词动作是一系列相似事件或状态的重复（如“首次亮相”）。
角色位置: 表示事件的参与者所处的位置，该事件中其他参与者不和他共处一处。
接受者: 从事件中获得

In [2]:
from unsloth import FastLanguageModel

max_seq_length = 5000
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="llama-3.2-3b-fsp-ft",
    model_name="unsloth/Qwen2.5-7B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Qwen2 patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

def formatting_prompts_func_test(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo[:-1], tokenize = False, add_generation_prompt = True) for convo in convos]
    return { "text" : texts, }

dataset = train_prompts.map(formatting_prompts_func, batched = True,)

test_prompts_df = pd.read_json('../fn1.7-test-prompts.jsonl', lines=True)
test_prompts = Dataset.from_pandas(test_prompts_df)

test_dataset = test_prompts.map(formatting_prompts_func_test, batched = True,)

Map: 100%|██████████| 6223/6223 [00:00<00:00, 25829.53 examples/s]


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2): 100%|██████████| 18865/18865 [00:10<00:00, 1809.64 examples/s]


In [6]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map: 100%|██████████| 18865/18865 [00:03<00:00, 4817.62 examples/s]


In [16]:
print(tokenizer.decode(trainer.train_dataset[5]["input_ids"]))

<|im_start|>system
### Task:
You are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. 

### Notes:
- Return the tagged sentence in a ```json ``` code block.
- Texts must not overlap.
<|im_end|>
<|im_start|>user
### Frame Information:
Frame Name: Natural_features
Frame Definition: The Locale is a geographical location as defined by shape. This frame includes natural geographic features, including land/ice forms and bodies of water.

Frame Elements:
Constituent_parts (Peripheral): Salient parts of the Locale.
Container_possessor (Extra-Thematic): The location that the Locale is a part of.
Formational_cause (Extra-Thematic): Indicates the action (or causer) which brings the f

In [14]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                                                                                                                                                                                                                                                                                               \n### Output: \n```json\n{'Type': 'prehistoric', 'Locale': 'lake'}\n```\n<|im_end|>\n"

In [17]:
import torch
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070 SUPER. Max memory = 11.994 GB.
7.52 GB of memory reserved.


In [10]:
# trainer_stats = trainer.train()
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 18,865 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,358
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.990500
2,1.913700
3,1.605600
4,1.444000
5,1.494000
6,1.574100
7,1.470100
8,1.101800
9,1.217000
10,0.804400


KeyboardInterrupt: 

In [22]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

predictions = []

for i, example in enumerate(test_dataset['messages']):
    inputs = tokenizer.apply_chat_template(
        example[:-1],
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    # print(inputs)
    print(tokenizer.decode(inputs[0]))
    outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True, streamer=text_streamer)
    # Decode the generated text, add to predictions, ignore prompt in output
    predictions.append(tokenizer.decode(outputs[0]).split("<|im_start|>assistant\n")[1])
    print(f"Completed {i+1} of {len(test_dataset['messages'])} generations.")

test_prompts_df['responses'] = predictions
test_prompts_df.to_json('qwen-2.5-7b-preds.jsonl', orient='records', lines=True)

<|im_start|>system
### Task:
You are given a sentence and a frame with its associated frame elements and sometimes examples. Your task is to label the frame elements in the sentence using JSON. Keys should only be one of the defined frame elements. Do not make up your own frame elements, and do not remove or change the input in any way. Identify the frame elements based on the highlighted target word. 

### Notes:
- Return the tagged sentence in a ```json ``` code block.
- Texts must not overlap.
<|im_end|>
<|im_start|>user
### Frame Information:
Frame Name: Calendric_unit
Frame Definition: Words in this frame name the different parts of the calendric cycle, both man-made and natural. The Unit (e.g. Tuesday) specifies some time period as part of a specific larger temporal Whole (Tuesday of next week), or may be resolved to an exact time span by a Relative_time (next Tuesday).

Frame Elements:
Relative_time (Core): Relative_time is used for the word or words that locate the time with re

KeyboardInterrupt: 

In [12]:
predictions

["### Output: \n```json\n{'Name': 'December'}\n```\n Prostitutes\natedRoute\nصندnection\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.Disclaimer\n.

In [ ]:
model.save_pretrained("llama-3.2-3b-fsp-ft2") # Local saving
tokenizer.save_pretrained("llama-3.2-3b-fsp-ft2")

In [22]:
x = test_prompts_df.head(len(predictions)).copy()
x['prediction'] = predictions
x.to_json('half-pred.jsonl', lines=True, orient='records')